In [1]:
!pip install transformers -q
!pip install torchview -q
!pip install torchviz -q

  Preparing metadata (setup.py) ... done


In [18]:
import torch
from torchviz import make_dot
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn

# Definir el modelo FineTuningBERT
class FineTuningBERT(nn.Module):
    def __init__(self, bert):
        super(FineTuningBERT, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(768, 64)
        self.ReLu = nn.ReLU()
        self.fc2 = nn.Linear(64, 93)  # Ajustar el número de clases

    def forward(self, sent_id, mask):
        outputs = self.bert(sent_id, attention_mask=mask)
        cls_hs = outputs['pooler_output']

        x = self.dropout(cls_hs)
        x = self.fc1(x)
        x = torch.nn.ReLU()(x)
        x = self.fc2(x)
        return x

# Cargar el modelo BERT base y tokenizer
bert_model = AutoModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Instantiate tu modelo fine-tuned BERT_Arch
model = FineTuningBERT(bert_model)

# Push el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Crear datos de ejemplo para la visualización
input_text = "cuales son los nombres de las lunas de urano"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
attention_mask = input_ids != tokenizer.pad_token_id

# Generar la visualización del modelo
dummy_output = model(input_ids, attention_mask)
model_graph = make_dot(dummy_output, params=dict(model.named_parameters()), show_attrs=False, show_saved=False)
model_graph.render('Fine-tuning-BERT', format='png', cleanup=True)


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'Fine-tuning-BERT.png'

In [25]:
import torch
from torchviz import make_dot
from transformers import AutoTokenizer
import torch.nn as nn

# Inicializar el tokenizador
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')


# Definir solo la capa FineTuningBERT
class FineTuningBERT(nn.Module):
    def __init__(self):
        super(FineTuningBERT, self).__init__()
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(768, 64)
        self.ReLu = nn.ReLU()
        self.fc2 = nn.Linear(64, 93)  # Ajustar el número de clases

    def forward(self, x):
        x = self.dropout(x)
        x = self.fc1(x)
        x = torch.nn.ReLU()(x)
        x = self.fc2(x)
        return x

# Crear una entrada simulada
dummy_input = torch.randn(1, 768)  # Tamaño de entrada similar a BERT

# Instanciar tu modelo FineTuningBERT
model1 = FineTuningBERT()

# Generar la visualización del modelo
dummy_output = model1(dummy_input)
model_graph = make_dot(dummy_output, params=dict(model1.named_parameters()), show_attrs=False, show_saved=False)
model_graph.render('Fine-tuning_only', format='png', cleanup=True)


'Fine-tuning_only.png'